### Import packages

In [1]:
import itertools
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from bayes_opt import BayesianOptimization

from utilities import cal_score, convert_types

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/__init__.py:45: FutureWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS will be built by the Apple Clang compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you won't need to install the gcc compiler anymore.
Instead of that, you'll need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", FutureWarning)


### Columns

In [2]:
cols_num = ['txn_dt', 'total_floor', 'building_complete_dt', 'parking_area', 'parking_price', 'txn_floor', 'land_area', 'building_area', 'lat', 'lon', 'village_income_median', 'town_population', 'town_area', 'town_population_density', 'doc_rate', 'master_rate', 'bachelor_rate', 'jobschool_rate', 'highschool_rate', 'junior_rate', 'elementary_rate', 'born_rate', 'death_rate', 'marriage_rate', 'divorce_rate', 'N_50', 'N_500', 'N_1000', 'N_5000', 'N_10000', 'I_10', 'I_50', 'I_100', 'I_250', 'I_500', 'I_1000', 'I_5000', 'I_10000', 'I_MIN', 'II_10', 'II_50', 'II_100', 'II_250', 'II_500', 'II_1000', 'II_5000', 'II_10000', 'II_MIN', 'III_10', 'III_50', 'III_100', 'III_250', 'III_500', 'III_1000', 'III_5000', 'III_10000', 'III_MIN', 'IV_10', 'IV_50', 'IV_100', 'IV_250', 'IV_500', 'IV_1000', 'IV_5000', 'IV_10000', 'IV_MIN', 'V_10', 'V_50', 'V_100', 'V_250', 'V_500', 'V_1000', 'V_5000', 'V_10000', 'V_MIN', 'VI_50', 'VI_100', 'VI_250', 'VI_500', 'VI_1000', 'VI_5000', 'VI_10000', 'VI_MIN', 'VII_10', 'VII_50', 'VII_100', 'VII_250', 'VII_500', 'VII_1000', 'VII_5000', 'VII_10000', 'VII_MIN', 'VIII_10', 'VIII_50', 'VIII_100', 'VIII_250', 'VIII_500', 'VIII_1000', 'VIII_5000', 'VIII_10000', 'VIII_MIN', 'IX_10', 'IX_50', 'IX_100', 'IX_250', 'IX_500', 'IX_1000', 'IX_5000', 'IX_10000', 'IX_MIN', 'X_10', 'X_50', 'X_100', 'X_250', 'X_500', 'X_1000', 'X_5000', 'X_10000', 'X_MIN', 'XI_10', 'XI_50', 'XI_100', 'XI_250', 'XI_500', 'XI_1000', 'XI_5000', 'XI_10000', 'XI_MIN', 'XII_10', 'XII_50', 'XII_100', 'XII_250', 'XII_500', 'XII_1000', 'XII_5000', 'XII_10000', 'XII_MIN', 'XIII_10', 'XIII_50', 'XIII_100', 'XIII_250', 'XIII_500', 'XIII_1000', 'XIII_5000', 'XIII_10000', 'XIII_MIN', 'XIV_10', 'XIV_50', 'XIV_100', 'XIV_250', 'XIV_500', 'XIV_1000', 'XIV_5000', 'XIV_10000', 'XIV_MIN']
cols_cat = ['building_material', 'city', 'building_type', 'building_use', 'parking_way', 'town', 'village']
cols_bin = ['I_index_50', 'I_index_500', 'I_index_1000', 'II_index_50', 'II_index_500', 'II_index_1000', 'III_index_50', 'III_index_500', 'III_index_1000', 'IV_index_50', 'IV_index_500', 'IV_index_1000', 'IV_index_5000', 'V_index_50', 'V_index_500', 'V_index_1000', 'VI_10', 'VI_index_50', 'VI_index_500', 'VI_index_1000', 'VII_index_50', 'VII_index_500', 'VII_index_1000', 'VIII_index_50', 'VIII_index_500', 'VIII_index_1000', 'IX_index_50', 'IX_index_500', 'IX_index_1000', 'IX_index_5000', 'X_index_50', 'X_index_500', 'X_index_1000', 'XI_index_50', 'XI_index_500', 'XI_index_1000', 'XI_index_5000', 'XI_index_10000', 'XII_index_50', 'XII_index_500', 'XII_index_1000', 'XIII_index_50', 'XIII_index_500', 'XIII_index_1000', 'XIII_index_5000', 'XIII_index_10000', 'XIV_index_50', 'XIV_index_500', 'XIV_index_1000']
cols_feats = cols_num + cols_cat + cols_bin

col_target = 'total_price'
col_target_log1p = 'target'

### Read data

In [3]:
df = pd.read_csv('dataset/train.csv', dtype=object)

In [4]:
df.columns

Index(['building_id', 'building_material', 'city', 'txn_dt', 'total_floor',
       'building_type', 'building_use', 'building_complete_dt', 'parking_way',
       'parking_area',
       ...
       'XIV_500', 'XIV_index_500', 'XIV_1000', 'XIV_index_1000', 'XIV_5000',
       'XIV_index_5000', 'XIV_10000', 'XIV_index_10000', 'XIV_MIN',
       'total_price'],
      dtype='object', length=235)

### Preprocessing

In [5]:
# Convert types
df = convert_types(df, cols_num, col_target=col_target)

# Generate feats (train-test-same feats)
#create_feats(df)

### Target transform

In [6]:
def target_transform(df):  # Add new columns inplace!!
    '''Transform the original target to the target for model to train on'''
    df['log_total_price'] = np.log1p(df['total_price'])
    return df

def target_inverse_transform(df, y_pred):
    '''Inverse transorform the model target to the original target of our problem'''
    y_pred_expm1 = np.expm1(y_pred)
    y_pred_final = np.clip(y_pred_expm1, 0, None)
    return y_pred_final            

In [7]:
df = target_transform(df)

In [8]:
# transformed target fit by the model
col_model_target = 'log_total_price'
#col_model_target = 'total_price'

### Feat engineering

In [9]:
from feature_engineering import CategoricalColumnsEncoder

In [10]:
class FeatureEngineering():
    def __init__(self):
        pass
    
    def fit_transform(self, df, cols_cat, cols_bin):
        df = df.copy()
        # Denote categorical-type
        self.cat_encoder = CategoricalColumnsEncoder(mode='pandas')
        self.cat_encoder.fit_transform(df, cols_cat+cols_bin)
        return df
    
    def transform(self, df):
        df = df.copy()
        self.cat_encoder.transform(df)
        return df

### Grid search

#### Prepare kfolds

In [11]:
n_folds = 3
feat_engs = []
df_trains = []
df_vals = []
folds = KFold(n_splits=n_folds, shuffle=True, random_state=123)
for i_fold, (itrain, ival) in enumerate(folds.split(df)): # kfold
    # split train, val
    df_train = df.loc[itrain]
    df_val = df.loc[ival]

    # feat eng
    feat_eng = FeatureEngineering()
    df_train = feat_eng.fit_transform(df_train, cols_cat, cols_bin)
    df_val = feat_eng.transform(df_val)
    
    feat_engs.append(feat_eng)
    df_trains.append(df_train)
    df_vals.append(df_val)

#### Bayesian Optimization

In [12]:
# grid search
params_fix = {'task': 'train',
              'boosting_type': 'gbdt',
              'objective': 'mse',
              'metric': 'mae',
              }
lgb_other_params = {'num_boost_round': 15000,
                    'verbose_eval': 2000,
                    'early_stopping_rounds': 1000,
                   }

In [13]:
int_params = ['num_leaves', 'min_data_in_leaf']  # list of params that needs to be int
def fix_params(params):
    """Fix the params into int"""
    for c in int_params:
        if c in params: 
            params[c] = int(round(params[c]))
    return params

In [14]:
def kfold_cv(**params_update):
    params_update = fix_params(params_update)
    params = params_fix.copy()
    params.update(params_update)
    
    print('params:', params)

    scores = []
    scores2 = []
    for i_fold, (df_train, df_val) in enumerate(zip(df_trains, df_vals)):
        print('==== Fold', i_fold+1, '====')

        # Construct lgb dataset
        lgb_train = lgb.Dataset(df_train[cols_feats], df_train[col_model_target]).construct()
        lgb_val = lgb.Dataset(df_val[cols_feats], df_val[col_model_target], reference=lgb_train).construct()

        model = lgb.train(params, lgb_train, valid_sets=lgb_val, **lgb_other_params)

        y_pred = model.predict(df_val[cols_feats])

        y_pred_final = target_inverse_transform(df_val, y_pred)

        score = mean_squared_log_error(df_val['total_price'], y_pred_final)
        score2 = cal_score(df_val['total_price'], y_pred_final)
        
        scores.append(score)
        scores2.append(score2)
    
    tuple_params = tuple(params.items())
    result = [tuple_params, np.mean(scores), scores, np.mean(scores2), scores2]
    gsearch_results.append(result)
    return result[3]

In [15]:
gsearch_results = []

In [16]:
from bayes_opt import BayesianOptimization

# Bounded region of parameter space
pbounds = {
    'learning_rate': (0.005, 0.05),
    'num_leaves': (63, 511),           # may reduce in dim-reduction exp
    'feature_fraction': (0.1, 1.0),
    'min_data_in_leaf': (5, 100),
    'lambda_l1': (0., 10.),
    'lambda_l2': (0., 10.),
}

optimizer = BayesianOptimization(
    f=kfold_cv,
    pbounds=pbounds,
    random_state=1,
)

In [17]:
%%time
optimizer.maximize(
    init_points=10,
    n_iter=65,
)

|   iter    |  target   | featur... | lambda_l1 | lambda_l2 | learni... | min_da... | num_le... |
-------------------------------------------------------------------------------------------------
params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mse', 'metric': 'mae', 'feature_fraction': 0.47531980423231657, 'lambda_l1': 7.203244934421581, 'lambda_l2': 0.0011437481734488664, 'learning_rate': 0.01860496576843279, 'min_data_in_leaf': 19, 'num_leaves': 104}
==== Fold 1 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.142325
Early stopping, best iteration is:
[2782]	valid_0's l1: 0.140793
==== Fold 2 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.143891
Early stopping, best iteration is:
[2639]	valid_0's l1: 0.14249
==== Fold 3 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.144017
Early stopping, best iteration is:
[2640]	valid_0's l1: 0.142873

[2000]	valid_0's l1: 0.144434
Early stopping, best iteration is:
[2627]	valid_0's l1: 0.143808
|  8        |  5.01e+03 |  0.3524   |  7.893    |  1.032    |  0.02516  |  91.32    |  194.5    |
params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mse', 'metric': 'mae', 'feature_fraction': 0.3589978047277139, 'lambda_l1': 1.3002857211827767, 'lambda_l2': 0.19366957870297075, 'learning_rate': 0.0355475989822951, 'min_data_in_leaf': 25, 'num_leaves': 182}
==== Fold 1 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.134502
[4000]	valid_0's l1: 0.13388
Early stopping, best iteration is:
[3145]	valid_0's l1: 0.13388
==== Fold 2 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.136396
[4000]	valid_0's l1: 0.135733
Early stopping, best iteration is:
[3171]	valid_0's l1: 0.135733
==== Fold 3 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.136043
[4000]	vali

[2000]	valid_0's l1: 0.1392
[4000]	valid_0's l1: 0.137956
Early stopping, best iteration is:
[3071]	valid_0's l1: 0.137956
==== Fold 3 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.139106
[4000]	valid_0's l1: 0.137906
Early stopping, best iteration is:
[3089]	valid_0's l1: 0.137906
|  15       |  5.269e+0 |  0.3798   |  2.936    |  9.947    |  0.03217  |  72.5     |  110.8    |
params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mse', 'metric': 'mae', 'feature_fraction': 0.47918778211114443, 'lambda_l1': 2.3827905183583886, 'lambda_l2': 0.205510238935388, 'learning_rate': 0.03082254199826657, 'min_data_in_leaf': 23, 'num_leaves': 181}
==== Fold 1 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.135446
Early stopping, best iteration is:
[2514]	valid_0's l1: 0.135027
==== Fold 2 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.137394
Early stopp

[4000]	valid_0's l1: 0.146509
[6000]	valid_0's l1: 0.143264
[8000]	valid_0's l1: 0.14188
[10000]	valid_0's l1: 0.141158
[12000]	valid_0's l1: 0.140726
[14000]	valid_0's l1: 0.140587
Did not meet early stopping. Best iteration is:
[15000]	valid_0's l1: 0.140547
|  22       |  5.25e+03 |  0.1766   |  1.603    |  1.565    |  0.009456 |  22.47    |  182.7    |
params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mse', 'metric': 'mae', 'feature_fraction': 0.7812962843865034, 'lambda_l1': 0.7696077962072345, 'lambda_l2': 5.299314054210699, 'learning_rate': 0.046851388414639684, 'min_data_in_leaf': 60, 'num_leaves': 376}
==== Fold 1 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.135248
[4000]	valid_0's l1: 0.135144
Early stopping, best iteration is:
[3298]	valid_0's l1: 0.135142
==== Fold 2 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.1375
[4000]	valid_0's l1: 0.137365
Early stopping, best 

==== Fold 2 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.138074
[4000]	valid_0's l1: 0.134969
[6000]	valid_0's l1: 0.134018
[8000]	valid_0's l1: 0.133618
Early stopping, best iteration is:
[7929]	valid_0's l1: 0.133618
==== Fold 3 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.139457
[4000]	valid_0's l1: 0.136221
[6000]	valid_0's l1: 0.135257
[8000]	valid_0's l1: 0.134961
Early stopping, best iteration is:
[7503]	valid_0's l1: 0.134961
|  30       |  5.431e+0 |  0.7478   |  1.164    |  0.2776   |  0.01102  |  20.52    |  182.4    |
params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mse', 'metric': 'mae', 'feature_fraction': 0.6255678697748331, 'lambda_l1': 0.022645669594920648, 'lambda_l2': 5.212124241670981, 'learning_rate': 0.023583475234125833, 'min_data_in_leaf': 60, 'num_leaves': 376}
==== Fold 1 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid

...

Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.135733
[4000]	valid_0's l1: 0.134088
[6000]	valid_0's l1: 0.133709
Early stopping, best iteration is:
[6268]	valid_0's l1: 0.133693
==== Fold 2 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.137601
[4000]	valid_0's l1: 0.136048
[6000]	valid_0's l1: 0.135685
Early stopping, best iteration is:
[6024]	valid_0's l1: 0.135684
==== Fold 3 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.137965
[4000]	valid_0's l1: 0.136429
[6000]	valid_0's l1: 0.136126
Early stopping, best iteration is:
[5820]	valid_0's l1: 0.136126
|  43       |  5.385e+0 |  0.3732   |  1.084    |  5.355    |  0.01857  |  59.3     |  377.0    |
params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mse', 'metric': 'mae', 'feature_fraction': 0.6844390719622487, 'lambda_l1': 0.1255526517159292, 'lambda_l2': 5.345637120419672, 'learning_rate': 0

[6000]	valid_0's l1: 0.135017
[8000]	valid_0's l1: 0.1342
[10000]	valid_0's l1: 0.133807
[12000]	valid_0's l1: 0.133535
[14000]	valid_0's l1: 0.133445
Early stopping, best iteration is:
[13058]	valid_0's l1: 0.133445
==== Fold 3 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.142966
[4000]	valid_0's l1: 0.137747
[6000]	valid_0's l1: 0.136053
[8000]	valid_0's l1: 0.13525
[10000]	valid_0's l1: 0.134837
[12000]	valid_0's l1: 0.134582
[14000]	valid_0's l1: 0.134477
Early stopping, best iteration is:
[13297]	valid_0's l1: 0.134477
|  50       |  5.46e+03 |  0.742    |  1.071    |  0.9567   |  0.006773 |  21.96    |  184.0    |
params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mse', 'metric': 'mae', 'feature_fraction': 0.7605247435616203, 'lambda_l1': 2.1637727960555218, 'lambda_l2': 1.3456994057464478, 'learning_rate': 0.014012695758105327, 'min_data_in_leaf': 27, 'num_leaves': 181}
==== Fold 1 ====
Training until validation scores 

Early stopping, best iteration is:
[4473]	valid_0's l1: 0.13415
|  56       |  5.442e+0 |  0.6343   |  0.8188   |  0.2464   |  0.02338  |  24.9     |  182.4    |
params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mse', 'metric': 'mae', 'feature_fraction': 0.46417095609933523, 'lambda_l1': 1.8554423380725893, 'lambda_l2': 0.3515921797794186, 'learning_rate': 0.023262221393601387, 'min_data_in_leaf': 22, 'num_leaves': 184}
==== Fold 1 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.134739
[4000]	valid_0's l1: 0.133324
Early stopping, best iteration is:
[3581]	valid_0's l1: 0.133324
==== Fold 2 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.137046
[4000]	valid_0's l1: 0.135571
Early stopping, best iteration is:
[3734]	valid_0's l1: 0.135571
==== Fold 3 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.136592
[4000]	valid_0's l1: 0.135274
Early st

params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mse', 'metric': 'mae', 'feature_fraction': 0.2512561353303957, 'lambda_l1': 0.3530760779675801, 'lambda_l2': 1.8119723962695067, 'learning_rate': 0.03890760890246386, 'min_data_in_leaf': 21, 'num_leaves': 183}
==== Fold 1 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.135668
[4000]	valid_0's l1: 0.13469
[6000]	valid_0's l1: 0.134567
Early stopping, best iteration is:
[5618]	valid_0's l1: 0.134567
==== Fold 2 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.137806
[4000]	valid_0's l1: 0.136828
[6000]	valid_0's l1: 0.136713
Early stopping, best iteration is:
[5675]	valid_0's l1: 0.136712
==== Fold 3 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.13894
[4000]	valid_0's l1: 0.137847
[6000]	valid_0's l1: 0.137713
Early stopping, best iteration is:
[5396]	valid_0's l1: 0.137713
|  64       |  5.33

[6000]	valid_0's l1: 0.134291
Early stopping, best iteration is:
[6939]	valid_0's l1: 0.134267
==== Fold 3 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.135898
[4000]	valid_0's l1: 0.135083
[6000]	valid_0's l1: 0.134935
Early stopping, best iteration is:
[6918]	valid_0's l1: 0.134915
|  70       |  5.422e+0 |  0.8183   |  0.08963  |  0.5492   |  0.03264  |  21.54    |  183.6    |
params: {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'mse', 'metric': 'mae', 'feature_fraction': 0.5016508809931852, 'lambda_l1': 0.15827732138988693, 'lambda_l2': 0.007948204157650629, 'learning_rate': 0.007511263398969944, 'min_data_in_leaf': 21, 'num_leaves': 184}
==== Fold 1 ====
Training until validation scores don't improve for 1000 rounds.
[2000]	valid_0's l1: 0.138873
[4000]	valid_0's l1: 0.133444
[6000]	valid_0's l1: 0.131559
[8000]	valid_0's l1: 0.130677
[10000]	valid_0's l1: 0.13017
[12000]	valid_0's l1: 0.129875
[14000]	valid_0's l1: 0.129717

In [18]:
optimizer.max

{'target': 5531.8638681913335,
 'params': {'feature_fraction': 0.5016508809931852,
  'lambda_l1': 0.15827732138988693,
  'lambda_l2': 0.007948204157650629,
  'learning_rate': 0.007511263398969944,
  'min_data_in_leaf': 20.91128754487751,
  'num_leaves': 183.84292613490783}}

In [19]:
gsearch_results.sort(key= lambda x: x[3], reverse=True)

In [20]:
gsearch_results

[[(('task', 'train'),
   ('boosting_type', 'gbdt'),
   ('objective', 'mse'),
   ('metric', 'mae'),
   ('feature_fraction', 0.5016508809931852),
   ('lambda_l1', 0.15827732138988693),
   ('lambda_l2', 0.007948204157650629),
   ('learning_rate', 0.007511263398969944),
   ('min_data_in_leaf', 21),
   ('num_leaves', 184)),
  0.04082014111696213,
  [0.039651531573588326, 0.04107584446536552, 0.04173304731193256],
  5531.8638681913335,
  [5548.865401079333, 5500.865334818009, 5545.860868676659]],
 [(('task', 'train'),
   ('boosting_type', 'gbdt'),
   ('objective', 'mse'),
   ('metric', 'mae'),
   ('feature_fraction', 0.4408045879394218),
   ('lambda_l1', 0.17919396658204745),
   ('lambda_l2', 1.0752647975229224),
   ('learning_rate', 0.005239307607920743),
   ('min_data_in_leaf', 22),
   ('num_leaves', 183)),
  0.040552913010630263,
  [0.03936291584824818, 0.04087006398201413, 0.041425759201628484],
  5520.53036558013,
  [5530.865273184758, 5489.865007374259, 5540.860816181375]],
 [(('task',

### Plots

In [ ]:
for param_name in pbounds:
    x_y = [[dict(row[0])[param_name], row[3]] for row in gsearch_results]
    x_y.sort(key=lambda xy: xy[0])
    x_y = np.array(x_y)
    plt.plot(x_y[:, 0], x_y[:, 1])
    plt.title(param_name); plt.ylabel('score')
    plt.show()

### Conclusion

- Round 1 best params (random seed = 123):
   ('task', 'train'),  
   ('boosting_type', 'gbdt'),  
   ('objective', 'mse'),  
   ('metric', 'mape'),  
   ('learning_rate', 0.015),  
   ('num_leaves', 255),  
   ('feature_fraction', 0.5),  
   ('min_data_in_leaf', 5)),  

Best scores: 5501.863278855861  
[5527.864888142274, 5498.864225711784, 5478.860722713526]

- Round 2 best params (random seed = 123):  
   ('task', 'train'),  
   ('boosting_type', 'gbdt'),  
   ('objective', 'mse'),  
   ('metric', 'mape'),  
   ('learning_rate', 0.015),  
   ('num_leaves', 255),  
   ('feature_fraction', 0.5),  
   ('min_data_in_leaf', 5),  
   ('lambda_l1', 0.1),  
   ('lambda_l2', 0)  
  
Best scores: *5515.529915334645*  
[5532.864763855017, 5514.864357360793, 5498.860624788125]

- Round 2 best params (random seed = 246):  
   ('task', 'train'),  
   ('boosting_type', 'gbdt'),  
   ('objective', 'mse'),  
   ('metric', 'mape'),  
   ('learning_rate', 0.015),  
   ('num_leaves', 255),  
   ('feature_fraction', 0.5),  
   ('min_data_in_leaf', 5),  
   ('lambda_l1', 0.1),  
   ('lambda_l2', 0.1)  
   
Best scores: *5524.863045737181*  
[5528.862755759428, 5530.862894274216, 5514.863487177898]

### Other info

In [21]:
lgb.plot_importance(model, figsize=(12, 30), height=0.5)
plt.show()

NameError: name 'model' is not defined

In [ ]:
lgb.plot_importance(model, figsize=(12, 30), height=0.5, importance_type='gain')
plt.show()